In [14]:
from collections import deque
import time
import requests

# Constants
BRAZIL = 'br'
EUROPE_NORDIC_EAST = 'eune'
EUROPE_WEST = 'euw'
KOREA = 'kr'
LATIN_AMERICA_NORTH = 'lan'
LATIN_AMERICA_SOUTH = 'las'
NORTH_AMERICA = 'na'
OCEANIA = 'oce'
RUSSIA = 'ru'
TURKEY = 'tr'

# Platforms
platforms = {
    BRAZIL: 'BR1',
    EUROPE_NORDIC_EAST: 'EUN1',
    EUROPE_WEST: 'EUW1',
    KOREA: 'KR',
    LATIN_AMERICA_NORTH: 'LA1',
    LATIN_AMERICA_SOUTH: 'LA2',
    NORTH_AMERICA: 'NA1',
    OCEANIA: 'OC1',
    RUSSIA: 'RU',
    TURKEY: 'TR1'
}

queue_types = [
    'CUSTOM',  # Custom games
    'NORMAL_5x5_BLIND',  # Normal 5v5 blind pick
    'BOT_5x5',  # Historical Summoners Rift coop vs AI games
    'BOT_5x5_INTRO',  # Summoners Rift Intro bots
    'BOT_5x5_BEGINNER',  # Summoner's Rift Coop vs AI Beginner Bot games
    'BOT_5x5_INTERMEDIATE',  # Historical Summoner's Rift Coop vs AI Intermediate Bot games
    'NORMAL_3x3',  # Normal 3v3 games
    'NORMAL_5x5_DRAFT',  # Normal 5v5 Draft Pick games
    'ODIN_5x5_BLIND',  # Dominion 5v5 Blind Pick games
    'ODIN_5x5_DRAFT',  # Dominion 5v5 Draft Pick games
    'BOT_ODIN_5x5',  # Dominion Coop vs AI games
    'RANKED_SOLO_5x5',  # Ranked Solo 5v5 games
    'RANKED_PREMADE_3x3',  # Ranked Premade 3v3 games
    'RANKED_PREMADE_5x5',  # Ranked Premade 5v5 games
    'RANKED_TEAM_3x3',  # Ranked Team 3v3 games
    'RANKED_TEAM_5x5',  # Ranked Team 5v5 games
    'BOT_TT_3x3',  # Twisted Treeline Coop vs AI games
    'GROUP_FINDER_5x5',  # Team Builder games
    'ARAM_5x5',  # ARAM games
    'ONEFORALL_5x5',  # One for All games
    'FIRSTBLOOD_1x1',  # Snowdown Showdown 1v1 games
    'FIRSTBLOOD_2x2',  # Snowdown Showdown 2v2 games
    'SR_6x6',  # Hexakill games
    'URF_5x5',  # Ultra Rapid Fire games
    'BOT_URF_5x5',  # Ultra Rapid Fire games played against AI games
    'NIGHTMARE_BOT_5x5_RANK1',  # Doom Bots Rank 1 games
    'NIGHTMARE_BOT_5x5_RANK2',  # Doom Bots Rank 2 games
    'NIGHTMARE_BOT_5x5_RANK5',  # Doom Bots Rank 5 games
    'ASCENSION_5x5',  # Ascension games
    'HEXAKILL',  # 6v6 games on twisted treeline
    'KING_PORO_5x5',  # King Poro game games
]

game_maps = [
    {'map_id': 1, 'name': "Summoner's Rift", 'notes': "Summer Variant"},
    {'map_id': 2, 'name': "Summoner's Rift", 'notes': "Autumn Variant"},
    {'map_id': 3, 'name': "The Proving Grounds", 'notes': "Tutorial Map"},
    {'map_id': 4, 'name': "Twisted Treeline", 'notes': "Original Version"},
    {'map_id': 8, 'name': "The Crystal Scar", 'notes': "Dominion Map"},
    {'map_id': 10, 'name': "Twisted Treeline", 'notes': "Current Version"},
    {'map_id': 11, 'name': "Summoner's Rift", 'notes': "Current Version"},
    {'map_id': 12, 'name': "Howling Abyss", 'notes': "ARAM Map"},
]

game_modes = [
    'CLASSIC',  # Classic Summoner's Rift and Twisted Treeline games
    'ODIN',  # Dominion/Crystal Scar games
    'ARAM',  # ARAM games
    'TUTORIAL',  # Tutorial games
    'ONEFORALL',  # One for All games
    'ASCENSION',  # Ascension games
    'FIRSTBLOOD',  # Snowdown Showdown games
    'KINGPORO',  # King Poro games
]

game_types = [
    'CUSTOM_GAME',  # Custom games
    'TUTORIAL_GAME',  # Tutorial games
    'MATCHED_GAME',  # All other games
]

sub_types = [
    'NONE',  # Custom games
    'NORMAL',  # Summoner's Rift unranked games
    'NORMAL_3x3',  # Twisted Treeline unranked games
    'ODIN_UNRANKED',  # Dominion/Crystal Scar games
    'ARAM_UNRANKED_5v5',  # ARAM / Howling Abyss games
    'BOT',  # Summoner's Rift and Crystal Scar games played against AI
    'BOT_3x3',  # Twisted Treeline games played against AI
    'RANKED_SOLO_5x5',  # Summoner's Rift ranked solo queue games
    'RANKED_TEAM_3x3',  # Twisted Treeline ranked team games
    'RANKED_TEAM_5x5',  # Summoner's Rift ranked team games
    'ONEFORALL_5x5',  # One for All games
    'FIRSTBLOOD_1x1',  # Snowdown Showdown 1x1 games
    'FIRSTBLOOD_2x2',  # Snowdown Showdown 2x2 games
    'SR_6x6',  # Hexakill games
    'CAP_5x5',  # Team Builder games
    'URF',  # Ultra Rapid Fire games
    'URF_BOT',  # Ultra Rapid Fire games against AI
    'NIGHTMARE_BOT',  # Nightmare bots
    'ASCENSION',  # Ascension games
    'HEXAKILL',  # Twisted Treeline 6x6 Hexakill
    ' KING_PORO',  # King Poro games
]

player_stat_summary_types = [
    'Unranked',  # Summoner's Rift unranked games
    'Unranked3x3',  # Twisted Treeline unranked games
    'OdinUnranked',  # Dominion/Crystal Scar games
    'AramUnranked5x5',  # ARAM / Howling Abyss games
    'CoopVsAI',  # Summoner's Rift and Crystal Scar games played against AI
    'CoopVsAI3x3',  # Twisted Treeline games played against AI
    'RankedSolo5x5',  # Summoner's Rift ranked solo queue games
    'RankedTeams3x3',  # Twisted Treeline ranked team games
    'RankedTeams5x5',  # Summoner's Rift ranked team games
    'OneForAll5x5',  # One for All games
    'FirstBlood1x1',  # Snowdown Showdown 1x1 games
    'FirstBlood2x2',  # Snowdown Showdown 2x2 games
    'SummonersRift6x6',  # Hexakill games
    'CAP5x5',  # Team Builder games
    'URF',  # Ultra Rapid Fire games
    'URFBots',  # Ultra Rapid Fire games played against AI
    'NightmareBot',  # Summoner's Rift games played against Nightmare AI
    'Hexakill',  # Twisted Treeline 6x6 Hexakill games
    'KingPoro',  # King Poro games
]

solo_queue, ranked_5s, ranked_3s = 'RANKED_SOLO_5x5', 'RANKED_TEAM_5x5', 'RANKED_TEAM_3x3'

api_versions = {
    'champion': 1.2,
    'current-game': 1.0,
    'featured-games': 1.0,
    'game': 1.3,
    'league': 2.5,
    'lol-static-data': 1.2,
    'lol-status': 1.0,
    'match': 2.2,
    'matchhistory': 2.2,
    'stats': 1.3,
    'summoner': 1.4,
    'team': 2.4
}


class LoLException(Exception):
    def __init__(self, error):
        self.error = error

    def __str__(self):
        return self.error


error_400 = LoLException("Bad request")
error_401 = LoLException("Unauthorized")
error_404 = LoLException("Game data not found")
error_429 = LoLException("Too many requests")
error_500 = LoLException("Internal server error")
error_503 = LoLException("Service unavailable")


def raise_status(response):
    if response.status_code == 400:
        raise error_400
    elif response.status_code == 401:
        raise error_401
    elif response.status_code == 404:
        raise error_404
    elif response.status_code == 429:
        raise error_429
    elif response.status_code == 500:
        raise error_500
    elif response.status_code == 503:
        raise error_503
    else:
        response.raise_for_status()


class RateLimit:
    def __init__(self, allowed_requests, seconds):
        self.allowed_requests = allowed_requests
        self.seconds = seconds
        self.made_requests = deque()

    def __reload(self):
        t = time.time()
        while len(self.made_requests) > 0 and self.made_requests[0] < t:
            self.made_requests.popleft()

    def add_request(self):
        self.made_requests.append(time.time() + self.seconds)

    def request_available(self):
        self.__reload()
        return len(self.made_requests) < self.allowed_requests


class RiotWatcher:
    def __init__(self, key, default_region=NORTH_AMERICA, limits=(RateLimit(10, 10), RateLimit(500, 600), )):
        self.key = key
        self.default_region = default_region
        self.limits = limits
        self.sleep_time = 1.5
        self.total_requests = 0.0
        self.failed_requests  = 0.0

    def can_make_request(self):
        for lim in self.limits:
            if not lim.request_available():
                return False
        return True

    def base_request(self, url, region, static=False, **kwargs):
        if region is None:
            region = self.default_region
        args = {'api_key': self.key}
        for k in kwargs:
            if kwargs[k] is not None:
                args[k] = kwargs[k]
        while(not self.can_make_request()):
            print("    Sleeping for 5 seconds to wait for request timelimit...")
            time.sleep(5)
        tries = 0
        while(True):
            try:
                r = requests.get(
                    'https://{proxy}.api.pvp.net/api/lol/{static}{region}/{url}'.format(
                        proxy='global' if static else region,
                        static='static-data/' if static else '',
                        region=region,
                        url=url
                    ),
                    params=args
                )
                if(r.status_code != 504 or tries >= 5):
                    break
                else:
                    tries += 1
                    time.sleep(5)
            except requests.exceptions.ConnectionError:
                tries += 1
                time.sleep(5)
        self.total_requests += 1
        if not static:
            for lim in self.limits:
                lim.add_request()
        #print("    Ran base request:  {0}".format('https://{proxy}.api.pvp.net/api/lol/{static}{region}/{url}'.format(proxy='global' if static else region,static='static-data/' if static else '',region=region,url=url)))
        if(r.status_code == 503):
            self.failed_requests += 1
            print("    503 ERROR! Retrying base_request:  {0}  {1}% failed".format(url, round(100.0*self.failed_requests/self.total_requests,2)))
            time.sleep(5)
            r = self.base_request_retry(url,region,static, **kwargs)
        raise_status(r)
        return r.json()

    def base_request_retry(self, url, region, static=False, **kwargs):
        """ Returns r instead of r.json() """
        if region is None:
            region = self.default_region
        args = {'api_key': self.key}
        for k in kwargs:
            if kwargs[k] is not None:
                args[k] = kwargs[k]
        tries = 0
        while(True): # To retry on GatewayTimeout errors
            try: # For Connection Error errors in module requests
                r = requests.get(
                    'https://{proxy}.api.pvp.net/api/lol/{static}{region}/{url}'.format(
                        proxy='global' if static else region,
                        static='static-data/' if static else '',
                        region=region,
                        url=url
                    ),
                    params=args
                )
                if(r.status_code != 504 or tries >= 5):
                    break
                else:
                    tries += 1
                    time.sleep(5)
            except requests.exceptions.ConnectionError:
                tries += 1
                time.sleep(5)
            self.total_requests += 1
            if not static:
                for lim in self.limits:
                    lim.add_request()
            if(r.status_code == 503): # For retrying on Service Unavailable errors (Riot stability problem)
                self.failed_requests += 1
                #print("    503 ERROR! Retrying base_request:  {0}  {1}% failed".format(url, round(100.0*self.failed_requests/self.total_requests,2)))
                time.sleep(3)
                r = self.base_request_retry(url,region,static, **kwargs)
            raise_status(r)
            return r

    def _observer_mode_request(self, url, proxy=None, **kwargs):
        if proxy is None:
            proxy = self.default_region
        args = {'api_key': self.key}
        for k in kwargs:
            if kwargs[k] is not None:
                args[k] = kwargs[k]
        r = requests.get(
            'https://{proxy}.api.pvp.net/observer-mode/rest/{url}'.format(
                proxy=proxy,
                url=url
            ),
            params=args
        )
        for lim in self.limits:
            lim.add_request()
        raise_status(r)
        return r.json()

    @staticmethod
    def sanitized_name(name):
        return name.replace(' ', '').lower()

    # champion-v1.2
    def _champion_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/champion/{end_url}'.format(
                version=api_versions['champion'],
                end_url=end_url
            ),
            region,
            **kwargs
        )

    def get_all_champions(self, region=None, free_to_play=False):
        return self._champion_request('', region, freeToPlay=free_to_play)

    def get_champion(self, champion_id, region=None):
        return self._champion_request('{id}'.format(id=champion_id), region)

    # current-game-v1.0
    def get_current_game(self, summoner_id, platform_id=None, region=None):
        if platform_id is None:
            platform_id = platforms[self.default_region]
        return self._observer_mode_request(
            'consumer/getSpectatorGameInfo/{platform}/{summoner_id}'.format(
                platform=platform_id,
                summoner_id=summoner_id
            ),
            region
        )

    # featured-game-v1.0
    def get_featured_games(self, proxy=None):
        return self._observer_mode_request('featured', proxy)

    # game-v1.3
    def _game_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/game/{end_url}'.format(
                version=api_versions['game'],
                end_url=end_url
            ),
            region,
            **kwargs
        )

    def get_recent_games(self, summoner_id, region=None):
        return self._game_request('by-summoner/{summoner_id}/recent'.format(summoner_id=summoner_id), region)

    # league-v2.5
    def _league_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/league/{end_url}'.format(
                version=api_versions['league'],
                end_url=end_url
            ),
            region,
            **kwargs
        )

    def get_league(self, summoner_ids=None, team_ids=None, region=None):
        """summoner_ids and team_ids arguments must be iterable, only one should be specified, not both"""
        if (summoner_ids is None) != (team_ids is None):
            if summoner_ids is not None:
                return self._league_request(
                    'by-summoner/{summoner_ids}'.format(summoner_ids=','.join([str(s) for s in summoner_ids])),
                    region
                )
            else:
                return self._league_request(
                    'by-team/{team_ids}'.format(team_ids=','.join([str(t) for t in team_ids])),
                    region
                )

    def get_league_entry(self, summoner_ids=None, team_ids=None, region=None):
        """summoner_ids and team_ids arguments must be iterable, only one should be specified, not both"""
        if (summoner_ids is None) != (team_ids is None):
            if summoner_ids is not None:
                return self._league_request(
                    'by-summoner/{summoner_ids}/entry'.format(
                        summoner_ids=','.join([str(s) for s in summoner_ids])
                    ),
                    region
                )
            else:
                return self._league_request(
                    'by-team/{team_ids}/entry'.format(team_ids=','.join([str(t) for t in team_ids])),
                    region
                )

    def get_challenger(self, region=None, queue=solo_queue):
        return self._league_request('challenger', region, type=queue)

    # lol-static-data-v1.2
    def _static_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/{end_url}'.format(
                version=api_versions['lol-static-data'],
                end_url=end_url
            ),
            region,
            static=True,
            **kwargs
        )

    def static_get_champion_list(self, region=None, locale=None, version=None, data_by_id=None, champ_data=None):
        return self._static_request(
            'champion',
            region,
            locale=locale,
            version=version,
            dataById=data_by_id,
            champData=champ_data
        )

    def static_get_champion(self, champ_id, region=None, locale=None, version=None, champ_data=None):
        return self._static_request(
            'champion/{id}'.format(id=champ_id),
            region,
            locale=locale,
            version=version,
            champData=champ_data
        )

    def static_get_item_list(self, region=None, locale=None, version=None, item_list_data=None):
        return self._static_request('item', region, locale=locale, version=version, itemListData=item_list_data)

    def static_get_item(self, item_id, region=None, locale=None, version=None, item_data=None):
        return self._static_request(
            'item/{id}'.format(id=item_id),
            region,
            locale=locale,
            version=version,
            itemData=item_data
        )

    def static_get_mastery_list(self, region=None, locale=None, version=None, mastery_list_data=None):
        return self._static_request(
            'mastery',
            region,
            locale=locale,
            version=version,
            masteryListData=mastery_list_data
        )

    def static_get_mastery(self, mastery_id, region=None, locale=None, version=None, mastery_data=None):
        return self._static_request(
            'mastery/{id}'.format(id=mastery_id),
            region,
            locale=locale,
            version=version,
            masteryData=mastery_data
        )

    def static_get_realm(self, region=None):
        return self._static_request('realm', region)

    def static_get_rune_list(self, region=None, locale=None, version=None, rune_list_data=None):
        return self._static_request('rune', region, locale=locale, version=version, runeListData=rune_list_data)

    def static_get_rune(self, rune_id, region=None, locale=None, version=None, rune_data=None):
        return self._static_request(
            'rune/{id}'.format(id=rune_id),
            region,
            locale=locale,
            version=version,
            runeData=rune_data
        )

    def static_get_summoner_spell_list(self, region=None, locale=None, version=None, data_by_id=None, spell_data=None):
        return self._static_request(
            'summoner-spell',
            region,
            locale=locale,
            version=version,
            dataById=data_by_id,
            spellData=spell_data
        )

    def static_get_summoner_spell(self, spell_id, region=None, locale=None, version=None, spell_data=None):
        return self._static_request(
            'summoner-spell/{id}'.format(id=spell_id),
            region,
            locale=locale,
            version=version,
            spellData=spell_data
        )

    def static_get_versions(self, region=None):
        return self._static_request('versions', region)

    # match-v2.2
    def _match_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/match/{end_url}'.format(
                version=api_versions['match'],
                end_url=end_url
            ),
            region,
            **kwargs
        )

    def get_match(self, match_id, region=None, include_timeline=False):
        return self._match_request(
            '{match_id}'.format(match_id=match_id),
            region,
            includeTimeline=include_timeline
        )

    # lol-status-v1.0
    @staticmethod
    def get_server_status(region=None):
        if region is None:
            url = 'shards'
        else:
            url = 'shards/{region}'.format(region=region)
        r = requests.get('http://status.leagueoflegends.com/{url}'.format(url=url))
        raise_status(r)
        return r.json()

    # match history-v2.2
    def _match_history_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/matchhistory/{end_url}'.format(
                version=api_versions['matchhistory'],
                end_url=end_url
            ),
            region,
            **kwargs
        )

    def get_match_history(self, summoner_id, region=None, champion_ids=None, ranked_queues=None, begin_index=None,
                          end_index=None):
        return self._match_history_request(
            '{summoner_id}'.format(summoner_id=summoner_id),
            region,
            championIds=champion_ids,
            rankedQueues=ranked_queues,
            beginIndex=begin_index,
            endIndex=end_index
        )

    # stats-v1.3
    def _stats_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/stats/{end_url}'.format(
                version=api_versions['stats'],
                end_url=end_url
            ),
            region,
            **kwargs
        )

    def get_stat_summary(self, summoner_id, region=None, season=None):
        return self._stats_request(
            'by-summoner/{summoner_id}/summary'.format(summoner_id=summoner_id),
            region,
            season='SEASON{}'.format(season) if season is not None else None)

    def get_ranked_stats(self, summoner_id, region=None, season=None):
        return self._stats_request(
            'by-summoner/{summoner_id}/ranked'.format(summoner_id=summoner_id),
            region,
            season='SEASON{}'.format(season) if season is not None else None
        )

    # summoner-v1.4
    def _summoner_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/summoner/{end_url}'.format(
                version=api_versions['summoner'],
                end_url=end_url
            ),
            region,
            **kwargs
        )

    def get_mastery_pages(self, summoner_ids, region=None):
        return self._summoner_request(
            '{summoner_ids}/masteries'.format(summoner_ids=','.join([str(s) for s in summoner_ids])),
            region
        )

    def get_rune_pages(self, summoner_ids, region=None):
        return self._summoner_request(
            '{summoner_ids}/runes'.format(summoner_ids=','.join([str(s) for s in summoner_ids])),
            region
        )

    def get_summoners(self, names=None, ids=None, region=None):
        if (names is None) != (ids is None):
            return self._summoner_request(
                'by-name/{summoner_names}'.format(
                    summoner_names=','.join([self.sanitized_name(n) for n in names])) if names is not None
                else '{summoner_ids}'.format(summoner_ids=','.join([str(i) for i in ids])),
                region
            )
        else:
            return None

    def get_summoner(self, name=None, _id=None, region=None):
        if (name is None) != (_id is None):
            if name is not None:
                name = self.sanitized_name(name)
                return self.get_summoners(names=[name, ], region=region)[name]
            else:
                return self.get_summoners(ids=[_id, ], region=region)[str(_id)]
        return None

    def get_summoner_name(self, summoner_ids, region=None):
        return self._summoner_request(
            '{summoner_ids}/name'.format(summoner_ids=','.join([str(s) for s in summoner_ids])),
            region
        )

    # team-v2.4
    def _team_request(self, end_url, region, **kwargs):
        return self.base_request(
            'v{version}/team/{end_url}'.format(
                version=api_versions['team'],
                end_url=end_url
            ),
            region,
            **kwargs
        )

    def get_teams_for_summoner(self, summoner_id, region=None):
        return self.get_teams_for_summoners([summoner_id, ], region=region)[str(summoner_id)]

    def get_teams_for_summoners(self, summoner_ids, region=None):
        return self._team_request(
            'by-summoner/{summoner_id}'.format(summoner_id=','.join([str(s) for s in summoner_ids])),
            region
        )

    def get_team(self, team_id, region=None):
        return self.get_teams([team_id, ], region=region)[str(team_id)]

    def get_teams(self, team_ids, region=None):
        return self._team_request('{team_ids}'.format(team_ids=','.join(str(t) for t in team_ids)), region)

In [27]:
get_match_history(self='b00c992b-6fc1-4795-abcc-7db85c1b72fa', summoner_id=20922259)


AttributeError: 'str' object has no attribute '_match_history_request'

In [21]:
import cPickle

def save_obj(obj, filename ):
    with open(filename, 'wb') as f:
        cPickle.dump(obj, f, cPickle.HIGHEST_PROTOCOL)

def load_obj(filename):
    with open(filename, 'rb') as f:
        return cPickle.load(f)

import sys
import time
from riotwatcher import RiotWatcher, NORTH_AMERICA, LoLException

def get_match_history(self, summoner_id, region=None, champion_ids=None, ranked_queues=None, begin_index=None, end_index=None):
    return self._match_history_request(
        '{summoner_id}'.format(summoner_id=summoner_id),
        region,
        championIds=champion_ids,
        rankedQueues=ranked_queues,
        beginIndex=begin_index,
        endIndex=end_index)

def secondsToStr(t):
    rediv = lambda ll,b : list(divmod(ll[0],b)) + ll[1:]
    return "%d:%02d:%02d.%03d" % tuple(reduce(rediv,[[t*1000,],1000,60,60]))

#key = #PUT YOUR API KEY HERE
w = RiotWatcher('b00c992b-6fc1-4795-abcc-7db85c1b72fa')

def get_participants_for_further_pulls(match):
    """ This function takes a riot api 'match' and returns only selected
        participant information. You may add to the returned information
        if you want.
        The return value is a list of dictionaries, one for each
        participant in the match. """
    participants = [ {} for i in range(len(match['participants'])) ]
    for i,p in enumerate(match['participants']):
        participants[i]['highestAchievedSeasonTier'] = p['highestAchievedSeasonTier']
        participants[i]['summonerId'] = match['participantIdentities'][p['participantId']-1]['player']['summonerId']
    return participants

def get_match_information_to_save(match):
    """ This function takes a riot api 'match' and parses
        through it to save only the information you want. You can
        choose to save everything, or only specific things.
        The return value is a dictionary containing only the
        information you want to save for the match. """
    # Uncomment the following line to save all the data
    return match
    data = {}
    # The below code is an example of specific data to save
    data['matchId'] = match['matchId'] # You MUST at least include this field!
    data['matchMode'] = match['matchMode']
    data['matchCreation'] = match['matchCreation']
    data['matchVersion'] = match['matchVersion']
    data['queueType'] = match['queueType']
    data['mapId'] = match['mapId']
    data['winningChamps'] = []
    data['losingChamps'] = []
    for p in match['participants']:
        if(p['stats']['winner']):
            data['winningChamps'].append(p['championId'])
        else:
            data['losingChamps'].append(p['championId'])
    return data

def parse_match_history(match_history):
    """ This function takes a summoner's match history and
        returns the matchIds from that match history along
        with the full list of matches. There is no reason
        to modify this function. """
    matchIds = [ match['matchId'] for match in match_history['matches'] ]
    return matchIds, match_history['matches']

def pull_this_match(match):
    """ This function takes a match from the *match history* request
        and returns True or False based on whether you want to 
        pull the match. If you want to pull every match, regardless
        of map type, etc. then just return True. """
    # This line parses the match's patch version into a usable format
    # For example, if the patch is 4.20.0.319 then
    # version == [4, 20, 0, 319]
    version = [int(x) for x in match['matchVersion'].split('.')]
    if(match['mapId'] == 11
        and match['queueType'] == 'RANKED_SOLO_5x5'):
        #and version[0] == 5 and version[1] >= 3):
        return True
    else:
        return False

def save_this_match(match):
    """ This function takes a match from the *match* request
        and returns True or False based on whether you want to
        save the match data. If you want to save every match
        then just return True. """
    # I will only save the match if there is a diamond player in it.
    save = False
    for p in match['participants']:
        if(p['highestAchievedSeasonTier'] == 'DIAMOND'):
            save = True
            break
    return save

def pull_this_participant(p):
    """ This function takes a participant from a match and
        returns True or False based on whether you want to 
        pull this summoner's data. """
    # I only want to pull the summoner if it is diamond tier.
    # I can only get highestAchievedSeasonTier  so I'll use that.
    if(p['highestAchievedSeasonTier'] == 'DIAMOND'):
        return True
    else:
        return False



def main():
    # BEFORE YOU RUN THIS FILE, MAKE A BACKUP OF ALL THE DATA IT'S GOING TO USE.
    # This data will be in one of the files opened below, so make copies
    # of all those files in case something goes wrong.

    try:
        if(sys.argv[1] == 'reset'):
            print("WARING: RESETTING ALL DATA IN 2 SECONDS!")
            print("You must manually re-copy your backed up version of unpulled_summoners.txt into that file.")
            print("The actual game data is not deleted, but it will be overwritten.")
            time.sleep(2)
            open('pulled_summoners.txt','w')
            open('pulled_matchIds.txt','w')
            open('paramfile.in','w').write('0')
    except:
        pass
    
    try:
        pulled_matchIds = open('pulled_matchIds.txt','r').readlines()
        pulled_matchIds = [int(x) for x in pulled_matchIds]
    except IOError:
        pulled_matchIds = []
    pulled_matchIds_file =  open('pulled_matchIds.txt','a')

    try:
        pulled_summoners = open('pulled_summoners.txt','r').readlines()
        pulled_summoners = [int(x) for x in pulled_summoners]
    except IOError:
        pulled_summoners = []
    pulled_summoners_file = open('pulled_summoners.txt','a')

    try:
        unpulled_summoners = open('unpulled_summoners.txt','r').readlines()
        unpulled_summoners = [int(x) for x in unpulled_summoners]
    except IOError:
        unpulled_summoners = []

    match_data_to_save = []
    these_pulled_summoners = []
    these_pulled_matches = []
    t_start = time.time()
    num_matches = 0.0
    num_pulled_summoners = 0.0
    pulled_matches = 0.0
    matched_you_actually_wanted = 0.0
    try:
        step = int(open('paramfile.in','r').readline())
    except IOError:
        step = 0
    while(True):
        # Start by getting a new summoner to pull matchID information for
        # If there is a summoner in unpulled_summoners, use that
        # If the unpulled_summoners list is empty, start at the beginning
        # of the summoners we have already pulled (their game data is
        # hopefully out of date by now).
        try:
            new_summoner = unpulled_summoners.pop()
        except IndexError:
            new_summoner = pulled_summoners.pop(0)
            print("Ran out of unpulled summoners, redoing a pulled summoner now. {0}".format(new_summoner))
        print("Getting match history from summoner: {0}".format(new_summoner))
        try:
            match_history = get_match_history(new_summoner,begin_index=0,end_index=15)
        except Exception,e:
            print("An ERROR occurred when pulling match history data for summonerId {0}! {1}".format(new_summoner,e))
            #print traceback.format_exc()
            continue
        try:
            matchIds, matches = parse_match_history(match_history)
        except KeyError,e:
            print("Some field you tried to access did not exist in the pulled summoner data: {0}".format(e))
            continue
        pulled_summoners.append(new_summoner)
        these_pulled_summoners.append(new_summoner)

        # Now we will pull the matches that were in this summoner's match history
        # if we decide it has data we want (see function pull_this_match)
        # If all the information you want is in the match history pull, you
        # can delete the pull request and modify the functions specified
        # however you want.
        for i,match in enumerate(matches):
            if(match['matchId'] not in pulled_matchIds and pull_this_match(match)):
                # Pull match
                print("  Getting match data for matchId {0}...".format(match['matchId']))
                try:
                    match_data = w.get_match(match['matchId'],include_timeline=False)
                except LoLException:
                    pulled_matchIds.append(match_data['matchId'])
                    these_pulled_matches.append(match_data['matchId'])
                    continue
                pulled_matches += 1.0
                # Append the matchId to the pulled matches list
                pulled_matchIds.append(match_data['matchId'])
                these_pulled_matches.append(match_data['matchId'])
                # Get participants in match
                participants = get_participants_for_further_pulls(match_data)
                # Add participants to unpulled summoners if necessary
                for p in participants:
                    # Unforunately, this if statement can take a while if there is enough
                    # data, but it's worth it not to pull data you already have.
                    if(pull_this_participant(p) and p['summonerId'] not in pulled_summoners and p['summonerId'] not in unpulled_summoners):
                        unpulled_summoners.append(p['summonerId'])
                # If the match information is what you want (see the save_this_match function)
                # then append the data you want to save to the list that we will save.
                if(save_this_match(match_data)):
                    # Save only specific information from each match (see the function)
                    match_data = get_match_information_to_save(match_data)
                    match_data_to_save.append(match_data)
                    num_matches += 1
                    matched_you_actually_wanted += 1
        # When we get 100+ games worth of new data, save them all to a new file.
        # This is nice because it prevents your files from becoming enormous,
        # and if you lose one you don't lose them all. It creates a bit more code
        # but I think it's worth it.
        # Note that there will not necessarily be exactly 100 games per file.
        if(num_matches >= 100):
            print("SAVING {0} MATCHES!".format(num_matches))
            # This saves the data as a cPickle. See Python's documentation for more info.
            save_obj(match_data_to_save,'game_data_{0}.pkl'.format(step))
            match_data_to_save = []
            num_matches = 0
            for mid in these_pulled_matches:
                pulled_matchIds_file.write("{0}\n".format(mid))
            these_pulled_matches = []
            # Save (append) the summonerIds we pulled to file
            for summoner in these_pulled_summoners:
                pulled_summoners_file.write("{0}\n".format(summoner))
            these_pulled_summoners = []
            # Save (overwrite) unpulled_summoners to file
            unpulled_summoners_file = open('unpulled_summoners.txt','w')
            for summoner in unpulled_summoners:
                unpulled_summoners_file.write("{0}\n".format(summoner))
            unpulled_summoners_file.close()
            # Increment step
            step += 1
            # Update the parameter file to the step number so that
            # if we restart this python program all our all data
            # does not get erased. Right now the only line in that file
            # is the step number.
            open('paramfile.in','w').write(str(step))
        num_pulled_summoners += 1.0
        print("  Approximate number of successfully pulled matches: {0}".format(pulled_matches))
        print("  Average time per (successful) match request: {0}".format((time.time()-t_start)/matched_you_actually_wanted))
        print("Number of successfully pulled summoners: {0}".format(num_pulled_summoners))
        print("Average time per (successfully pulled) summoner: {0}".format((time.time()-t_start)/num_pulled_summoners))





if __name__ == '__main__':
    main()

Getting match history from summoner: 35630174
An ERROR occurred when pulling match history data for summonerId 35630174! get_match_history() takes at least 2 arguments (3 given)
Getting match history from summoner: 28750301
An ERROR occurred when pulling match history data for summonerId 28750301! get_match_history() takes at least 2 arguments (3 given)
Getting match history from summoner: 20922259
An ERROR occurred when pulling match history data for summonerId 20922259! get_match_history() takes at least 2 arguments (3 given)
Getting match history from summoner: 253631
An ERROR occurred when pulling match history data for summonerId 253631! get_match_history() takes at least 2 arguments (3 given)
Getting match history from summoner: 22989674
An ERROR occurred when pulling match history data for summonerId 22989674! get_match_history() takes at least 2 arguments (3 given)
Getting match history from summoner: 35651858
An ERROR occurred when pulling match history data for summonerId 356

IndexError: pop from empty list